In [1]:
import os
import torch
import pickle
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses, SentencesDataset, InputExample, models
from transformers import AutoTokenizer, AutoModel
import os
import pickle
import torch
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import euclidean
from transformers import AutoTokenizer, AutoModel

/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
num_gpus = torch.cuda.device_count()

In [3]:
num_gpus

4

In [4]:
def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy().flatten()


def load_finetuned_model(checkpoint_path):
    device = torch.device('cuda' if num_gpus > 0 else 'cpu')
    model = AutoModel.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    model = model.to(device)
    return model , tokenizer , device


def get_embeddings(texts):
    embeddings = [get_embedding(text) for text in texts]
    return np.vstack(embeddings)


def print_euclidean_distances(addresses, triplets, num_triplets=10):
    subset_triplets = triplets[:num_triplets]
    for i, (a, p, n, _) in enumerate(subset_triplets):
        
        anchor_emb = get_embedding(addresses[a])
        positive_emb = get_embedding(addresses[p])
        negative_emb = get_embedding(addresses[n])

        pos_distance = euclidean(anchor_emb, positive_emb)
        neg_distance = euclidean(anchor_emb, negative_emb)

        print(f"Triplet {i+1}:")
        print(subset_triplets[i])
        print(f"  Anchor-Positive Distance: {pos_distance}")
        print(f"  Anchor-Negative Distance: {neg_distance}")
        print(f"  Difference: {neg_distance - pos_distance}\n")




In [5]:
input_pickle_path = 'DataSet_triplet/PUNJAB_05_without_city.pkl'

with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

triplets = data.get('triplets', [])
addresses = data.get('addresses', [])
lat_lngs = data.get('lat_lngs', [])

print(f"Length of triplets: {len(triplets)}, {triplets[-1]}")
print(f"Length of addresses: {len(addresses)}, {addresses[-1]}")
print(f"Length of lat_lngs: {len(lat_lngs)}, {lat_lngs[-1]}")

Length of triplets: 3217904, (1114168, 1114167, 1114099, 3)
Length of addresses: 1114180, wardno 1 vpo sarhala ranuan
Length of lat_lngs: 1114180, [31.2044125, 75.8583784]


In [11]:
triplets = triplets[:10000]
len(triplets)

10000

In [13]:
final_model_save_path = '/home/uddeshya.singh/Experiments/punjab_roberta_model_with_city_10_new_loss_check/checkpoint_epoch53'
model, tokenizer, device = load_finetuned_model(final_model_save_path)
num_triplets_to_use = 50
print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 9.931182861328125
  Anchor-Negative Distance: 10.85823917388916
  Difference: 0.9270563125610352

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 14.295516014099121
  Anchor-Negative Distance: 15.174806594848633
  Difference: 0.8792905807495117

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 9.931182861328125
  Anchor-Negative Distance: 7.222484588623047
  Difference: -2.708698272705078

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 4.099771022796631
  Anchor-Negative Distance: 15.387921333312988
  Difference: 11.288150310516357

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 4.099771022796631
  Anchor-Negative Distance: 19.5696964263916
  Difference: 15.46992540359497

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 4.099771022796631
  Anchor-Negative Distance: 34.709129333496094
  Difference: 30.609358310699463

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 4.15021657943

In [13]:
final_model_save_path = '/home/uddeshya.singh/DataSet_triplet/Roberta_classification_r'
model, tokenizer, device = load_finetuned_model(final_model_save_path)
num_triplets_to_use = 10
print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/DataSet_triplet/Roberta_classification_r and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 25.43716812133789
  Anchor-Negative Distance: 48.97944259643555
  Difference: 23.542274475097656

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 49.818443298339844
  Anchor-Negative Distance: 53.48967742919922
  Difference: 3.671234130859375

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 25.43716812133789
  Anchor-Negative Distance: 38.753456115722656
  Difference: 13.316287994384766

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 22.086732864379883
  Anchor-Negative Distance: 35.00202560424805
  Difference: 12.915292739868164

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 22.086732864379883
  Anchor-Negative Distance: 37.42295455932617
  Difference: 15.336221694946289

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 22.086732864379883
  Anchor-Negative Distance: 36.571895599365234
  Difference: 14.485162734985352

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 26.5465145

In [38]:
final_model_save_path = "/home/uddeshya.singh/Experiments/results_new_2/final_model"
model, tokenizer, device = load_finetuned_model(final_model_save_path)

num_triplets_to_use = 10

print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/uddeshya.singh/Experiments/results_new_2/final_model and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 13.32730484008789
  Anchor-Negative Distance: 17.5146427154541
  Difference: 4.187337875366211

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 20.320056915283203
  Anchor-Negative Distance: 21.53264617919922
  Difference: 1.2125892639160156

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 13.32730484008789
  Anchor-Negative Distance: 11.578231811523438
  Difference: -1.7490730285644531

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 12.3921537399292
  Anchor-Negative Distance: 15.570472717285156
  Difference: 3.178318977355957

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 12.3921537399292
  Anchor-Negative Distance: 12.550565719604492
  Difference: 0.15841197967529297

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 12.3921537399292
  Anchor-Negative Distance: 18.21770668029785
  Difference: 5.825552940368652

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 9.33618736267089

In [7]:
checkpoint_path =  '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch4'
model, tokenizer , device = load_finetuned_model(checkpoint_path)
num_triplets_to_use = 10 
print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 7.453205108642578
  Anchor-Negative Distance: 19.514848709106445
  Difference: 12.061643600463867

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 21.223440170288086
  Anchor-Negative Distance: 32.96896743774414
  Difference: 11.745527267456055

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 7.453205108642578
  Anchor-Negative Distance: 21.091716766357422
  Difference: 13.638511657714844

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 8.946653366088867
  Anchor-Negative Distance: 35.59333038330078
  Difference: 26.646677017211914

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 8.946653366088867
  Anchor-Negative Distance: 43.992820739746094
  Difference: 35.04616737365723

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 8.946653366088867
  Anchor-Negative Distance: 25.479764938354492
  Difference: 16.533111572265625

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 9.800099372

In [16]:
checkpoint_path =  '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch3'
model, tokenizer , device = load_finetuned_model(checkpoint_path)

num_triplets_to_use = 10 

print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 6.339777946472168
  Anchor-Negative Distance: 16.8538818359375
  Difference: 10.514103889465332

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 14.419986724853516
  Anchor-Negative Distance: 25.78009033203125
  Difference: 11.360103607177734

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 6.339777946472168
  Anchor-Negative Distance: 20.35637855529785
  Difference: 14.016600608825684

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 7.767730712890625
  Anchor-Negative Distance: 29.721263885498047
  Difference: 21.953533172607422

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 7.767730712890625
  Anchor-Negative Distance: 42.86494445800781
  Difference: 35.09721374511719

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 7.767730712890625
  Anchor-Negative Distance: 25.786544799804688
  Difference: 18.018814086914062

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 8.118219375610

In [15]:
checkpoint_path = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"  
model, tokenizer , device = load_finetuned_model(checkpoint_path)
num_triplets_to_use = 10 
print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 13.05056095123291
  Anchor-Negative Distance: 17.442262649536133
  Difference: 4.391701698303223

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 18.448692321777344
  Anchor-Negative Distance: 20.446468353271484
  Difference: 1.9977760314941406

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 13.05056095123291
  Anchor-Negative Distance: 10.921241760253906
  Difference: -2.129319190979004

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 13.755464553833008
  Anchor-Negative Distance: 17.424671173095703
  Difference: 3.6692066192626953

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 13.755464553833008
  Anchor-Negative Distance: 12.580591201782227
  Difference: -1.1748733520507812

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 13.755464553833008
  Anchor-Negative Distance: 16.334575653076172
  Difference: 2.579111099243164

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 10.582

In [13]:
###### 05 dataset , base model pan india !!
checkpoint_path =  '/home/devanapalli.ravi/disk_1/paper/data/untrained_models/distilroberta-base/'
model, tokenizer , device = load_finetuned_model(checkpoint_path)

num_triplets_to_use = 10 

print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 2.677131175994873
  Anchor-Negative Distance: 2.795302629470825
  Difference: 0.11817145347595215

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 2.657477617263794
  Anchor-Negative Distance: 2.7020514011383057
  Difference: 0.04457378387451172

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 2.677131175994873
  Anchor-Negative Distance: 1.816903829574585
  Difference: -0.8602273464202881

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 1.8531798124313354
  Anchor-Negative Distance: 2.2056684494018555
  Difference: 0.35248863697052

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 1.8531798124313354
  Anchor-Negative Distance: 3.0756373405456543
  Difference: 1.2224575281143188

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 1.8531798124313354
  Anchor-Negative Distance: 2.2815513610839844
  Difference: 0.4283715486526489

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 1.54754

In [12]:
checkpoint_path =  '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch1'
model, tokenizer , device = load_finetuned_model(checkpoint_path)
num_triplets_to_use = 25 
print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 9.830082893371582
  Anchor-Negative Distance: 13.537705421447754
  Difference: 3.707622528076172

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 16.264301300048828
  Anchor-Negative Distance: 19.80118179321289
  Difference: 3.5368804931640625

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 9.830082893371582
  Anchor-Negative Distance: 14.859084129333496
  Difference: 5.029001235961914

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 10.816303253173828
  Anchor-Negative Distance: 25.06682777404785
  Difference: 14.250524520874023

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 10.816303253173828
  Anchor-Negative Distance: 38.63608932495117
  Difference: 27.819786071777344

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 10.816303253173828
  Anchor-Negative Distance: 24.427236557006836
  Difference: 13.610933303833008

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 9.35310268

In [12]:
checkpoint_path =  '/home/uddeshya.singh/Experiments/punjab_roberta_model_without_city_05/checkpoint_epoch2'
model, tokenizer , device = load_finetuned_model(checkpoint_path)

num_triplets_to_use = 25 

print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 5.535243034362793
  Anchor-Negative Distance: 13.720959663391113
  Difference: 8.18571662902832

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 12.247666358947754
  Anchor-Negative Distance: 19.267929077148438
  Difference: 7.020262718200684

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 5.535243034362793
  Anchor-Negative Distance: 14.667210578918457
  Difference: 9.131967544555664

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 8.271035194396973
  Anchor-Negative Distance: 25.32159996032715
  Difference: 17.050564765930176

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 8.271035194396973
  Anchor-Negative Distance: 41.292964935302734
  Difference: 33.02192974090576

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 8.271035194396973
  Anchor-Negative Distance: 26.66520118713379
  Difference: 18.394165992736816

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 10.139650344848

In [16]:
checkpoint_path = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"  

model, tokenizer , device = load_finetuned_model(checkpoint_path)

num_triplets_to_use = 10 

print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Some weights of RobertaModel were not initialized from the model checkpoint at /home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Triplet 1:
(4, 5, 37, 1)
  Anchor-Positive Distance: 5.5187482833862305
  Anchor-Negative Distance: 10.738306999206543
  Difference: 5.2195587158203125

Triplet 2:
(4, 5, 88, 3)
  Anchor-Positive Distance: 5.5187482833862305
  Anchor-Negative Distance: 11.98876953125
  Difference: 6.4700212478637695

Triplet 3:
(5, 4, 37, 1)
  Anchor-Positive Distance: 5.5187482833862305
  Anchor-Negative Distance: 11.747516632080078
  Difference: 6.228768348693848

Triplet 4:
(5, 4, 88, 3)
  Anchor-Positive Distance: 5.5187482833862305
  Anchor-Negative Distance: 12.55126953125
  Difference: 7.0325212478637695

Triplet 5:
(12, 112753, 112195, 1)
  Anchor-Positive Distance: 14.605476379394531
  Anchor-Negative Distance: 14.191181182861328
  Difference: -0.4142951965332031

Triplet 6:
(12, 112753, 111744, 2)
  Anchor-Positive Distance: 14.605476379394531
  Anchor-Negative Distance: 14.931740760803223
  Difference: 0.3262643814086914

Triplet 7:
(12, 112753, 111642, 3)
  Anchor-Positive Distance: 14.6054

In [20]:
checkpoint_path =  '/home/uddeshya.singh/Experiments/punjab_roberta_model2.1/checkpoint_epoch_Con1'
model, tokenizer , device = load_finetuned_model(checkpoint_path)

num_triplets_to_use = 10 

print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(4, 5, 37, 1)
  Anchor-Positive Distance: 2.7805049419403076
  Anchor-Negative Distance: 68.16820526123047
  Difference: 65.38770031929016

Triplet 2:
(4, 5, 88, 3)
  Anchor-Positive Distance: 2.7805049419403076
  Anchor-Negative Distance: 76.01651000976562
  Difference: 73.23600506782532

Triplet 3:
(5, 4, 37, 1)
  Anchor-Positive Distance: 2.7805049419403076
  Anchor-Negative Distance: 67.51615905761719
  Difference: 64.73565411567688

Triplet 4:
(5, 4, 88, 3)
  Anchor-Positive Distance: 2.7805049419403076
  Anchor-Negative Distance: 75.37237548828125
  Difference: 72.59187054634094

Triplet 5:
(12, 112753, 112195, 1)
  Anchor-Positive Distance: 7.370152473449707
  Anchor-Negative Distance: 73.11762237548828
  Difference: 65.74746990203857

Triplet 6:
(12, 112753, 111744, 2)
  Anchor-Positive Distance: 7.370152473449707
  Anchor-Negative Distance: 58.97348403930664
  Difference: 51.603331565856934

Triplet 7:
(12, 112753, 111642, 3)
  Anchor-Positive Distance: 7.3701524734

## Few more epochs on triplet fine-tuning

In [22]:
checkpoint_path =  '/home/uddeshya.singh/Experiments/punjab_roberta_model2.2/checkpoint_epoch_Con21'
model, tokenizer , device = load_finetuned_model(checkpoint_path)

num_triplets_to_use = 10 

print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(4, 5, 37, 1)
  Anchor-Positive Distance: 3.1960039138793945
  Anchor-Negative Distance: 88.83289337158203
  Difference: 85.63688945770264

Triplet 2:
(4, 5, 88, 3)
  Anchor-Positive Distance: 3.1960039138793945
  Anchor-Negative Distance: 87.25013732910156
  Difference: 84.05413341522217

Triplet 3:
(5, 4, 37, 1)
  Anchor-Positive Distance: 3.1960039138793945
  Anchor-Negative Distance: 88.45953369140625
  Difference: 85.26352977752686

Triplet 4:
(5, 4, 88, 3)
  Anchor-Positive Distance: 3.1960039138793945
  Anchor-Negative Distance: 86.51628875732422
  Difference: 83.32028484344482

Triplet 5:
(12, 112753, 112195, 1)
  Anchor-Positive Distance: 13.864480972290039
  Anchor-Negative Distance: 86.1375503540039
  Difference: 72.27306938171387

Triplet 6:
(12, 112753, 111744, 2)
  Anchor-Positive Distance: 13.864480972290039
  Anchor-Negative Distance: 74.01277923583984
  Difference: 60.148298263549805

Triplet 7:
(12, 112753, 111642, 3)
  Anchor-Positive Distance: 13.86448097

In [17]:
checkpoint_path =  '/home/uddeshya.singh/Experiments/punjab_roberta_model2.1/checkpoint_epoch_Con1'
model, tokenizer , device = load_finetuned_model(checkpoint_path)

num_triplets_to_use = 10 

print_euclidean_distances(addresses, triplets, num_triplets=num_triplets_to_use)

Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 24.600444793701172
  Anchor-Negative Distance: 40.038177490234375
  Difference: 15.437732696533203

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 14.643144607543945
  Anchor-Negative Distance: 15.712746620178223
  Difference: 1.0696020126342773

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 24.600444793701172
  Anchor-Negative Distance: 21.64607048034668
  Difference: -2.954374313354492

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 8.871150970458984
  Anchor-Negative Distance: 59.755699157714844
  Difference: 50.88454818725586

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 8.871150970458984
  Anchor-Negative Distance: 51.927982330322266
  Difference: 43.05683135986328

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 8.871150970458984
  Anchor-Negative Distance: 55.0084228515625
  Difference: 46.137271881103516

Triplet 7:
(110, 442119, 117, 1)
  Anchor-Positive Distance: 15.31193065